In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import mylib as ml

# 데이터 로딩 예시 (데이터프레임 형식)
df = ml.data_000660()

c:\big18\dl-dev\dl-dev\project\mylib.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={


In [2]:
# 시퀀스 길이 및 특성 수 설정
time_steps = 5  # 사용할 시퀀스 길이
features = 2     # 종가와 거래량

# 데이터 준비 함수
def prepare_data(df, time_steps):
    X = []
    y = []
    
    for i in range(len(df) - time_steps):
        X.append(df.iloc[i:i + time_steps][['close', 'volume']].values)
        # 다음 종가가 현재 종가보다 높으면 1, 낮으면 0
        y.append(1 if df.iloc[i + time_steps]['close'] > df.iloc[i + time_steps - 1]['close'] else 0)

    return np.array(X), np.array(y)

# 1D CNN 모델 정의
def create_cnn_model(input_shape):
    cnn_input = layers.Input(shape=input_shape)
    x = layers.Conv1D(filters=32, kernel_size=2, activation='relu')(cnn_input)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    cnn_output = layers.Dense(64, activation='relu')(x)
    return models.Model(inputs=cnn_input, outputs=cnn_output)

# 트랜스포머 모델 정의
class TransformerBlock(layers.Layer):
    def __init__(self, embed_size, heads, dropout=0):
        super(TransformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=heads, key_dim=embed_size)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.feed_forward = models.Sequential([
            layers.Dense(2048, activation='relu'),
            layers.Dense(embed_size)
        ])
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)

    def call(self, x):
        attention = self.attention(x, x)
        x = self.norm1(x + self.dropout1(attention))
        feed_forward = self.feed_forward(x)
        return self.norm2(x + self.dropout2(feed_forward))

# 결합 모델 생성
def create_combined_model(input_shape):
    cnn_model = create_cnn_model(input_shape)
    
    # CNN 출력과 트랜스포머 입력
    transformer_input = layers.Input(shape=(time_steps, features))  # 시퀀스 길이에 맞춰 조정
    transformer_block = TransformerBlock(embed_size=features, heads=4)(transformer_input)
    transformer_output = layers.GlobalAveragePooling1D()(transformer_block)
    
    # CNN 모델의 입력과 출력
    cnn_output = cnn_model.output
    cnn_model_input = cnn_model.input
    
    # 두 모델의 출력 결합
    combined = layers.Concatenate()([cnn_output, transformer_output])
    combined_output = layers.Dense(1, activation='sigmoid')(combined)  # 다음 종가 예측
    
    return models.Model(inputs=[cnn_model_input, transformer_input], outputs=combined_output)


In [3]:
# 모델 생성
input_shape_cnn = (time_steps, features)
combined_model = create_combined_model(input_shape_cnn)

# 모델 컴파일
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약
combined_model.summary()

# 데이터 준비
X, y = prepare_data(df, time_steps)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 2)]       0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 4, 32)        160         ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 2, 32)        0           ['conv1d[0][0]']                 
                                                                                                  
 input_2 (InputLayer)           [(None, 5, 2)]       0           []                               
                                                                                            

In [4]:
data=pd.DataFrame(y)
distribution = data[0].value_counts()
print(distribution)

0
0    12178
1     5207
Name: count, dtype: int64


In [ ]:
# 모델 훈련
combined_model.fit([X, X], y, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=3, min_lr=1e-6)
combined_model.fit([X, X], y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[reduce_lr])
